In [ ]:
# также нужен pytorch 
https://pytorch.org/

In [1]:
!pip install llama_index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-langchain
!pip install load_dotenv
!pip install faiss-cpu
!pip install bitsandbytes
!pip install langchain_huggingface
!pip install bitsandbytes

!pip install llama-index-vector-stores-faissv
!pip install huggingface_hub

  Using cached llama_index-0.12.24-py3-none-any.whl.metadata (12 kB)
  Using cached llama_index_agent_openai-0.4.6-py3-none-any.whl.metadata (727 bytes)
  Using cached llama_index_cli-0.4.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.12.24.post1-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.6.9-py3-none-any.whl.metadata (3.6 kB)
  Using cached llama_index_llms_openai-0.3.25-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_multi_modal_llms_openai-0.4.3-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_program_openai-0.3.1-py3-none-any.whl.metadata (764 bytes)
  Using cached llama_index_question_gen_openai-0.3.0-py3-none-any.whl.metadata (783 bytes)
  Using cached llama_index_readers_file-0.4.6-py3-none-any.whl.metadata (5.4 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metad

ERROR: Could not find a version that satisfies the requirement llama-index-vector-stores-faissv (from versions: none)
ERROR: No matching distribution found for llama-index-vector-stores-faissv


In [1]:
from huggingface_hub import login
from dotenv import load_dotenv
import openai
import os
import torch
from torch import bfloat16
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig, pipeline
import bitsandbytes
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import StorageContext
from langchain_huggingface  import HuggingFaceEmbeddings
from llama_index.llms.huggingface import HuggingFaceLLM

In [2]:
# download .env
load_dotenv("../.env")

try:
    HF_TOKEN= os.getenv('TOKEN_HF')
    OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')

    if HF_TOKEN is None:
        raise ValueError("TOKEN_HF не найдена!")

    if OPENAI_API_KEY is None:
        raise ValueError("OPENAI_API_KEY не найдена!")

    login(HF_TOKEN)
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    print("OpenAI API key successful installed!")
except Exception as e:
    print(f"Ошибка: {e}")

OpenAI API key successful installed!


# Модель

In [3]:
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
# MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

In [4]:
DEFAULT_SYSTEM_PROMPT = """Ты — MLTeacherBot, интеллектуальный помощник и учитель математики для машинного обучения.
Твоя задача — объяснять сложные математические темы понятным языком, помогать решать задачи и давать полезные советы.

### Твой стиль общения:
- Дружелюбный, терпеливый, мотивирующий.
- Объясняй просто и структурированно, подстраиваясь под уровень пользователя.
- Используй аналогии и примеры.

Ты — наставник, который делает обучение увлекательным и доступным.
"""

In [5]:
import torch
from torch import bfloat16
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig, pipeline
import bitsandbytes


# 4-разрядная конфигурация для загрузки LLM с меньшим объемом памяти графического процессора
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Использую 4-битную квантизацию
    bnb_4bit_quant_type='nf4',  # используем формат NF4
    bnb_4bit_use_double_quant=True,  # применить повторное квантовани
    bnb_4bit_compute_dtype=bfloat16  # Тип из которого преобразуем (доступная точность модели)
)

# Загружаем модель с квантизацией и автоматическим распределением
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto"
)

model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [6]:
def messages_to_prompt(messages):
    prompt = ""

    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"


    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<s>bot\n"
    return prompt

def completion_to_prompt(completion):
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

In [7]:
llm = HuggingFaceLLM(
    model=model,             # модель
    model_name=MODEL_NAME,   # идентификатор модели
    tokenizer=tokenizer,     # токенизатор
    # max_new_tokens=generation_config.max_new_tokens, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
    max_new_tokens = 500,
    model_kwargs={"quantization_config": quantization_config}, # параметры квантования
    generate_kwargs = {   # параметры для инференса
      "bos_token_id": generation_config.bos_token_id, # токен начала последовательности
      "eos_token_id": generation_config.eos_token_id, # токен окончания последовательности
      "pad_token_id": generation_config.pad_token_id, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
      "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
      "repetition_penalty": generation_config.repetition_penalty,
      "temperature": generation_config.temperature,
      "do_sample": True,
      "top_k": generation_config.top_k,
      "top_p": generation_config.top_p
    },
    messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
    completion_to_prompt=completion_to_prompt, # функции для генерации текста
    device_map="auto",                         # автоматически определять устройство
)

# подготовка бызы знаний

## Загружаем документы

In [8]:
# !gdown "https://drive.google.com/uc?export=download&id=1QQEday3YxXd8b9mcDzlpPkI2i8W75Cvn"

In [9]:
# !unzip "/content/library.zip"

In [10]:
from llama_index.core import SimpleDirectoryReader

#Загрузка всех Pdf из папки
Documents = SimpleDirectoryReader("../library").load_data()

In [11]:
expected_files = [
    "shevcov_lineynaya_algebra.pdf",
    "Математический анализ (В. А. Зорич) 1 часть.pdf",
    "Математический анализ (В. А. Зорич) 2 часть.pdf",
    "Введение в математическую статистику_Ивченко и др_2010 -600с_.pdf",
    "ТЕОРИЮ ВЕРОЯТНОСТЕЙ.pdf",
    "мат статистика.pdf"
]

# Получаем имена загруженных файлов
loaded_files = [doc.metadata.get("file_name", "unknown") for doc in Documents]

# Проверяем, все ли файлы загружены
missing_files = set(expected_files) - set(loaded_files)
if missing_files:
    print(f"Предупреждение: следующие файлы не были загружены: {missing_files}")
else:
    print("Все файлы успешно загружены.")

Предупреждение: следующие файлы не были загружены: {'Введение в математическую статистику_Ивченко и др_2010 -600с_.pdf', 'мат статистика.pdf', 'Математический анализ (В. А. Зорич) 2 часть.pdf', 'Математический анализ (В. А. Зорич) 1 часть.pdf', 'ТЕОРИЮ ВЕРОЯТНОСТЕЙ.pdf'}


In [12]:
import numpy as np
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [13]:
# import numpy as np
# from llama_index.embeddings.langchain import LangchainEmbedding
# from langchain_huggingface  import HuggingFaceEmbeddings

# embed_model = LangchainEmbedding(
#   HuggingFaceEmbeddings(
#       model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
#       # model_kwargs={"device": "cpu"}
#   )
# )

In [14]:
# Настройка ServiceContext (глобальная настройка параметров LLM)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

# Создаём FAISS индекс

In [15]:
# Создаем простое графовое хранилище
graph_store = SimpleGraphStore()

# Устанавливаем информацию о хранилище в StorageContext
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [16]:
from llama_index.core import KnowledgeGraphIndex

# Запускаем генерацию индексов из документа с помощью KnowlegeGraphIndex
indexKG = KnowledgeGraphIndex.from_documents(documents=Documents,          # данные для построения графов
                                           max_triplets_per_chunk=3,        # сколько обработывать триплетов связей для каждого блока данных
                                           show_progress=True,              # показывать процесс выполнения
                                           include_embeddings=True,         # включение векторных вложений в индекс для расширенной аналитики
                                           storage_context=storage_context) # куда сохранять результаты

Parsing nodes:   0%|          | 0/576 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/576 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/6 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/11 [00:00<?, ?it/s]

In [25]:
indexKG.storage_context.persist(persist_dir="./saved_index")  # Указываем путь для сохранения


In [22]:
from pyvis.network import Network
from IPython.display import display, HTML
import IPython

# Получаем граф
g = indexKG.get_networkx_graph(500)

# Создаём интерактивный граф
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

# Сохраняем HTML-файл с указанием кодировки UTF-8
html_content = net.html  # Получаем HTML-код
with open("graph.html", "w", encoding="utf-8") as f:
    f.write(html_content)

# Открываем граф в Jupyter Notebook
display(HTML("graph.html"))

In [23]:
import webbrowser
webbrowser.open("graph.html")


True

In [24]:
from llama_index.core import load_index_from_storage, StorageContext

# Создаём StorageContext из сохранённой папки
storage_context = StorageContext.from_defaults(persist_dir="./kg_index")

# Загружаем KnowledgeGraphIndex
indexKG = load_index_from_storage(storage_context)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/tinat/Documents/Project_Oleg/model/kg_index/docstore.json'